In [5]:
%config IPCompleter.greedy=True
import re
import json
import requests
import random
import time
import pandas as pd
from itertools import chain
from tqdm import tqdm_notebook as tqdm
from xml.etree import ElementTree as elemTree

Extract only question forms (run once and do not run anymore :) )

In [24]:
# with open('properties_forms.txt', 'r') as inf:
#     res = {}
#     pid = None
#     aliases = []
#     read_flag = False
#     for line in inf:
#         line = line.strip('\n')
#         if line.startswith('Aliases'):
#             read_flag = False
#         elif line.startswith('Question'):
#             read_flag = True
#         if line.startswith('Count') or \
#            line.startswith('Description') or \
#            line.startswith('Aliases') or \
#            line.startswith('Question') or \
#            line.startswith('Label') or \
#            line.startswith('# ') or \
#            not line:
#                 continue
#         elif line.startswith('####'):
#             res[pid] = aliases
#             pid = None
#             aliases = []
#         elif line.startswith('PID'):
#             pid = line.replace('PID: ', '')
#         elif read_flag:
#             aliases.append(line)
            
# with open('properties_suggest_aliases.json', 'w') as ouf:
#     json.dump(res, ouf, ensure_ascii=False, indent=4)

Read aliases

In [53]:
with open('properties_suggest_aliases.json', 'r') as inf:
    prop_aliases = json.load(inf)

Convert re to words

In [54]:
def convert_alias(alias):
    if '*' not in alias:
        return [alias]
    if 'как.*' not in alias:
        while '.* ' in alias:
            alias = alias.replace('.* ', ' ')
        while '.*' in alias:
            alias = alias.replace('.*', ' ')
        return [alias]
    res = []
    res.append(alias.replace('как.*', 'какой'))
    res.append(alias.replace('как.*', 'какая'))
    res.append(alias.replace('как.*', 'какое'))
    res.append(alias.replace('как.*', 'какие'))
    res.append(alias.replace('как.*', 'каким'))
    res.append(alias.replace('как.*', 'какими'))
    res.append(alias.replace('как.*', 'какого'))
    res.append(alias.replace('как.*', 'какую'))
    res.append(alias.replace('как.*', 'каких'))
    res.append(alias.replace('как.*', 'каком'))
    for i in range(len(res)):
        while '.* ' in res[i]:
            res[i] = res[i].replace('.* ', ' ')
        while '.*' in res[i]:
            res[i] = res[i].replace('.*', ' ')
    return res

def convert_prop_aliases(pa):
    new_pa = {}
    for p in pa:
        new_pa[p] = []
        for a in pa[p]:
            new_pa[p].extend(convert_alias(a))
    return new_pa

In [55]:
prop_aliases = convert_prop_aliases(prop_aliases)

In [56]:
prop_aliases

{'P577': ['когда выпущен',
  'когда издан',
  'когда состоялась премьера',
  'когда впервые'],
 'P1545': ['какой по счету',
  'какая по счету',
  'какое по счету',
  'какие по счету',
  'каким по счету',
  'какими по счету',
  'какого по счету',
  'какую по счету',
  'каких по счету',
  'каком по счету',
  'какой по порядку',
  'какая по порядку',
  'какое по порядку',
  'какие по порядку',
  'каким по порядку',
  'какими по порядку',
  'какого по порядку',
  'какую по порядку',
  'каких по порядку',
  'каком по порядку',
  'какой номер'],
 'P478': ['в каком томе', 'какой том'],
 'P17': ['в какой стране', 'в каком государстве'],
 'P131': [],
 'P21': ['какой пол у', 'какого пола'],
 'P106': ['кто по профессии', 'какая профессия', 'чем занимается'],
 'P921': ['о чем', 'про что', 'какая тема', 'что описывает'],
 'P569': ['когда день рождения', 'когда родил', 'в каком году родил'],
 'P735': ['как зовут'],
 'P27': ['какое гражданство', 'где живет'],
 'P50': ['какой создал',
  'какая создал'

In [61]:
len(list(chain.from_iterable(prop_aliases.values())))

897

In [57]:
q_beginning = 'в каком создан'
resp = requests.get(url=f'https://suggest.yandex.ru/suggest-ya.cgi?v=4&part={q_beginning}')
print(resp.json())

['в каком создан', ['в каком стиле выполнен уилтонский диптих созданный в конце 14 века', 'в каком году была создана международная любительская федерация борьбы', 'в каком году была создана международная лыжная комиссия', 'в каком году была создана международная федерация гандбола', 'в каком году был создан олимпийский комитет ссср', 'в каком году был создан ютуб', 'в каком году был создан комитет по правам ребенка', 'в каком году был опубликован императорский манифест о создании в россии государственной думы', 'в каком городе было создано второе ополчение в период смуты', 'в каком году был создан майнкрафт'], {'r': 11148, 'log': 'sgtype:WWWWWWWWWW'}]


Read PIDs

In [64]:
with open('pids.json', 'r') as inf:
    pids = json.load(inf)

Send requests to suggest

In [77]:
all_responses = []

In [80]:
for prop, name in tqdm(pids[10:100]):
    for q_beginning in prop_aliases[prop]:
        resp = requests.get(url=f'https://suggest.yandex.ru/suggest-ya.cgi?v=4&part={q_beginning}')
        all_responses.append((prop, q_beginning, resp.json()))
        
        sleep_time = random.randint(10, 20)
        time.sleep(sleep_time)
    with open('responses_log.json', 'w') as ouf:
        json.dump(all_responses, ouf, ensure_ascii=False, indent=4)

In [83]:
all_responses

[('P577',
  'когда выпущен',
  ['когда выпущен',
   ['когда выпущен аккумулятор как узнать',
    'когда выпущена трудовая книжка тк-vi',
    'когда выпущена первая русская печатная книга',
    'когда выпущена первая печатная книга',
    'когда выпущена первая датированная книга славянской печати',
    'когда выпущен айфон',
    'когда выпущен первый айфон',
    'когда выпущен айфон 7',
    'когда выпущен айфон 6',
    'когда выпущен айфон se'],
   {'r': 11148, 'log': 'sgtype:BBTTTTTTTT', 'continue': ' аккумулятор'}]),
 ('P577',
  'когда издан',
  ['когда издан',
   ['когда издана конституция рф',
    'когда издана жалованная грамота',
    'когда издан приказ 227 ни шагу назад',
    'когда издана первая печатная книга',
    'когда издан указ президента рф о создании вс россии',
    'когда издана русская правда',
    'когда издан словарь даля',
    'когда издана красная книга',
    'когда издан детский альбом чайковского',
    'когда издан манифест о вольности дворянской'],
   {'r': 1114

Process suggest answers

In [100]:
pid_col = []
beg_col = []
sug_col = []
for pid, beginning, resp in all_responses:
    for sugg in resp[1]:
        if type(sugg) != list:
            pid_col.append(pid)
            beg_col.append(beginning)
            sug_col.append(sugg)

suggest_df = pd.DataFrame(data={'pid': pid_col, 'beginning': beg_col, 'suggestion': sug_col}, columns=['pid', 'beginning', 'suggestion'])
suggest_df.to_csv('suggest_questions_from_top100_properties.csv', index=False)

In [99]:
suggest_df

,pid,beginning,suggestion
0,P577,когда выпущен,когда выпущен аккумулятор как узнать
1,P577,когда выпущен,когда выпущена трудовая книжка тк-vi
2,P577,когда выпущен,когда выпущена первая русская печатная книга
3,P577,когда выпущен,когда выпущена первая печатная книга
4,P577,когда выпущен,когда выпущена первая датированная книга славя...
5,P577,когда выпущен,когда выпущен айфон
6,P577,когда выпущен,когда выпущен первый айфон
7,P577,когда выпущен,когда выпущен айфон 7
8,P577,когда выпущен,когда выпущен айфон 6
9,P577,когда выпущен,когда выпущен айфон se


In [103]:
suggest_df_unique = suggest_df.drop_duplicates(subset=['suggestion'], keep='first')

In [104]:
suggest_df_unique.shape

(7033, 3)

In [42]:
q_beginning = 'кто лермонтов'

In [43]:
resp_google = requests.get(url=f'http://suggestqueries.google.com/complete/search?output=toolbar&hl=en&q={q_beginning}')
tree = elemTree.fromstring(resp_google.content.decode('utf-8', 'ignore'))

In [44]:
q_list = []
for child in tree.iter('suggestion'):
    q_list.append(child.attrib['data'])

In [45]:
q_list

['кто лермонтов по национальности',
 'кто лермонтов',
 'кто лермонтов по гороскопу',
 'кто лермонтова убил',
 'кто лермонтова мцыри',
 'лермонтов кто убил',
 'лермонтов кто это',
 'лермонтов кто такой',
 'лермонтов кто он',
 'лермонтов кто по нации']